# Import Libraries

In [1]:
%load_ext rpy2.ipython

C:\Users\Areeka Aijaz\Anaconda3\lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  "The symbol '%s' is not in this R namespace/package." % name


In [2]:
%%R
library(dplyr)
library(ggplot2)
library("readxl")
library(tidyr)
library(corrplot)
library(tidyverse)
library(superml)
library(mlbench)
library(caret)
library(gridExtra)

R[write to console]: 
Attaching package: 'dplyr'


R[write to console]: The following objects are masked from 'package:stats':

    filter, lag


R[write to console]: The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


R[write to console]: corrplot 0.88 loaded

R[write to console]: -- Attaching packages --------------------------------------- tidyverse 1.3.0 --

R[write to console]: v tibble  2.1.3     v stringr 1.4.0
v readr   1.3.1     v forcats 0.4.0
v purrr   0.3.3     

R[write to console]: -- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

R[write to console]: Loading required package: R6

R[write to console]: Loading required package: lattice

R[write to console]: 
Attaching package: 'caret'


R[write to console]: The following object is masked from 'package:purrr':

    lift


R[write to console]: 
Attaching package: 'gridExtra

In [3]:
import pandas as pd
import numpy as np

import fuzzywuzzy
from fuzzywuzzy import process

from nltk.stem import PorterStemmer

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedShuffleSplit

from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from sklearn.ensemble import VotingClassifier

from tkinter import *
from tkinter import Toplevel, Button, Tk, Menu, messagebox, Label 
from tkinter import ttk
import random

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

C:\Users\Areeka Aijaz\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Utility Functions

## Extract Data

In [4]:
%%R
read_data <- function(sheet){
    data <- read_excel("Data/StudentDataset.xlsx", sheet=sheet)
    data
}

## Data Transformation

In [5]:
%%R
student_Preprocessing <- function(student_data){
  student_data <- student_data[, c("STUDENT_ID", "GENDER", "BATCH", "CAMPUS", "PROG_CODE", "WARNING", "CGPA", 
                                  "FIRST_SEM", "LAST_SEM", "STATUS", "CITY", "SSC_BOARD", "SSC_OBTAINED", 
                                  "HSSC_BOARD", "HSSC_OBTAINED", "OLEVEL_BOARD", "OLEVEL_OBTAINED",
                                  "ALEVEL_BOARD", "ALEVEL_OBTAINED")]

  student_data$GENDER <- as.factor(student_data$GENDER)
  student_data$BATCH <- as.factor(student_data$BATCH)
  student_data$CAMPUS <- as.factor(student_data$CAMPUS)
  student_data$PROG_CODE <- as.factor(student_data$PROG_CODE)
  student_data$FIRST_SEM <- as.factor(student_data$FIRST_SEM)
  student_data$LAST_SEM <- as.factor(student_data$LAST_SEM)
  student_data$STATUS <- as.factor(student_data$STATUS)
  student_data$CITY <- as.factor(student_data$CITY)

  student_data$WARNING <- suppressWarnings(as.numeric(student_data$WARNING))
  student_data$CGPA <- suppressWarnings(as.numeric(student_data$CGPA))

  student_data <- student_data %>% distinct()
  student_data <- student_data[!duplicated(student_data[, "STUDENT_ID"]), ]

  SSC_OLEVEL <- vector(mode = "character", length = nrow(student_data))
  School <- vector(mode = "character", length = nrow(student_data))
  Sec_Grade <- vector(mode = "numeric", length = nrow(student_data))

  HSSC_ALEVEL <- vector(mode = "character", length = nrow(student_data))
  College <- vector(mode = "character", length = nrow(student_data))
  High_Sec_Grade <- vector(mode = "numeric", length = nrow(student_data))

  student_data$SECONDARY <- SSC_OLEVEL
  student_data$SCHOOL <- School
  student_data$SEC_GRADE <- Sec_Grade

  student_data$HIGHER_SECONDARY <- HSSC_ALEVEL
  student_data$COLLEGE <- College
  student_data$HIG_SEC_GRADE <- High_Sec_Grade

  for (i in 1:nrow(student_data)){
    if (!is.na(student_data[i, 13])){
      student_data[i, 20] <- "SSC"
      student_data[i, 21] <- student_data[i, 12]
      student_data[i ,22] <- student_data[i, 13]
    }
    else if (!is.na(student_data[i, 17])) {
      student_data[i, 20] <- "OLEVEL"
      student_data[i, 21] <- student_data[i, 16]
      student_data[i ,22] <- student_data[i, 17]    
    }
    else {
      student_data[i, 20] <- NA
      student_data[i, 21] <- NA
      student_data[i ,22] <- NA     
    }
  
  
    if (!is.na(student_data[i, 15])){
      student_data[i, 23] <- "HSSC"
      student_data[i, 24] <- student_data[i, 14]
      student_data[i ,25] <- student_data[i, 15]
    }
    else if (!is.na(student_data[i, 19])) {
      student_data[i, 23] <- "ALEVEL"
      student_data[i, 24] <- student_data[i, 18]
      student_data[i ,25] <- student_data[i, 19]    
    }
    else {
      student_data[i, 23] <- NA
      student_data[i, 24] <- NA
      student_data[i ,25] <- NA     
    }
  }

  student_data <- student_data[ , c("STUDENT_ID", "GENDER", "BATCH", "CAMPUS", "PROG_CODE", "WARNING", "FIRST_SEM", 
                                    "LAST_SEM", "STATUS", "CITY", "SECONDARY", "SCHOOL", "SEC_GRADE", "HIGHER_SECONDARY",
                                    "COLLEGE", "HIG_SEC_GRADE", "CGPA")]
    
  student_data
    
}

In [6]:
%%R
semester_Preprocessing <- function(semester_data){

    colnames(semester_data) <- c("Semester", "Student_Id", "SGPA", "CGPA", "CoreCount", "ElectiveCount")

    split <- function(x, Tail = TRUE){ temp <- unlist(strsplit(x, " ")); if (Tail) tail(temp,1) else head(temp,1)}

    semester_data$Year <- sapply(semester_data$Semester, FUN = split)
                                      
    semester_data$Session <- sapply(semester_data$Semester, FUN = split, Tail = FALSE)

    semester_data <- semester_data[, c("Student_Id", "Semester", "Session", "Year", "SGPA", "CGPA", "CoreCount", "ElectiveCount")]
    
    sem_seq <- c("Spring", "Summer", "Fall")
                                      
    semester_data <- semester_data[order(factor(semester_data$Session, levels = sem_seq)), ]
    
    semester_data <- semester_data %>% arrange(semester_data$Year)
        
    semester_data <- subset(semester_data, select = c("Student_Id", "Semester", "SGPA", "CGPA"))
    
    unique_semester <- unique(semester_data$Semester)

    unique_ids <- unique(semester_data$Student_Id)
    
    semester_details <- c("SEM_1_SGPA", "SEM_1_CGPA", "SEM_2_SGPA", "SEM_2_CGPA", "SEM_3_SGPA", "SEM_3_CGPA", 
                          "SEM_4_SGPA", "SEM_4_CGPA", "SEM_5_SGPA", "SEM_5_CGPA", "SEM_6_SGPA", "SEM_6_CGPA",
                          "SEM_7_SGPA", "SEM_7_CGPA", "SEM_8_SGPA", "SEM_8_CGPA", "SEM_9_SGPA", "SEM_9_CGPA", 
                          "SEM_10_SGPA", "SEM_10_CGPA", "SEM_11_SGPA", "SEM_11_CGPA", "SEM_12_SGPA", "SEM_12_CGPA",
                          "SEM_13_SGPA", "SEM_13_CGPA", "SEM_14_SGPA", "SEM_14_CGPA", "SEM_15_SGPA", "SEM_15_CGPA")
    
    data <- data.frame(matrix(ncol = length(semester_details), nrow = length(unique_ids)))
    colnames(data) <- semester_details
    
    data[semester_details] <- sapply(data[semester_details], as.character)
            
    data = cbind(STUDENT_ID = unique_ids, data)
    
    data$STUDENT_ID = as.character(data$STUDENT_ID)                

    Total_Sem <- vector(mode = "integer", length = nrow(data))
     
    data$TOTAL_SEM <- Total_Sem

    for (i in 1:nrow(data)){
      z <- data[i,1] == semester_data[ ,1]
      row_ <- which(z)
      data[i, 32] <- length(z[z==TRUE])
      for (j in 1:15){
        data[i, j*2] <- semester_data[row_[j], 3]
        data[i, (j*2)+1] <- semester_data[row_[j], 4]
      }
    }
    
    data[, 2:31] <- suppressWarnings(sapply(data[, 2:31], as.numeric))
                                      
    data
}

In [7]:
%%R
course_Preprocessing <- function(CourseData1,CourseData2,StudentData) 
{
  Columns <- c("STUDENT_ID" , "CODE" , "TITLE" ,"RELATION_ID" , "GRADE")
  CourseData <- rbind(subset(CourseData1, select = Columns), subset(CourseData2, select = Columns))

  Get_CD <- function(x){substring(x , first = 0 , last = 2)}
  CourseData$COURSE_DOMAIN <- sapply(CourseData$CODE , FUN = Get_CD)

  CourseData$CODE <- sapply(CourseData$CODE , FUN = toupper)
  CourseData$TITLE <- sapply(CourseData$TITLE , FUN = tolower)
  CourseData$RELATION_ID <- sapply(CourseData$RELATION_ID , FUN = trimws)

  levels <- c( 'A+'="4.00", 'A'="4.00", 'A-'="3.67", 'B+'="3.33", "B"="3.00", "B-" ="2.67",
              "C+"="2.33", "C"="2.00", "C-"="1.67","D+"="1.33", "D"="1.00")
  CourseData <- transform(CourseData, COURSE_GRADE = levels[as.character(GRADE)])

  CourseData <- filter(CourseData , CourseData$RELATION_ID == "C") %>% select(-RELATION_ID)

  CourseData <- filter(CourseData , CourseData$COURSE_DOMAIN != "ME")
  CourseData <- filter(CourseData , CourseData$COURSE_DOMAIN != "ML")
  CourseData <- filter(CourseData , CourseData$COURSE_DOMAIN != "NL")
  CourseData <- filter(CourseData , CourseData$COURSE_DOMAIN != "NS")
  CourseData <- filter(CourseData , CourseData$COURSE_DOMAIN != "SL")

  CourseData <- filter(CourseData , CourseData$GRADE != "F")
  CourseData <- filter(CourseData , CourseData$GRADE != "W")
  CourseData <- filter(CourseData , CourseData$GRADE != "F/R")
  CourseData <- filter(CourseData , CourseData$GRADE != "FA")
  CourseData <- filter(CourseData , CourseData$GRADE != "S")
  CourseData <- filter(CourseData , CourseData$GRADE != "LL")
  CourseData <- filter(CourseData , CourseData$GRADE != "L1")
  CourseData <- filter(CourseData , CourseData$GRADE != "L2")
  CourseData <- filter(CourseData , CourseData$GRADE != "L3")
  CourseData <- filter(CourseData , CourseData$GRADE != "I")

  CourseData <- CourseData %>% select(-c(GRADE, CODE))

  CourseData <- distinct(CourseData)
  CourseData <- CourseData %>% distinct(STUDENT_ID , TITLE , .keep_all = TRUE)

  CourseData$COURSE_GRADE <- as.numeric(CourseData$COURSE_GRADE )
  CourseData$COURSE_DOMAIN <- as.factor(CourseData$COURSE_DOMAIN)

  ProgramCode <- StudentData
  ProgramCode <- subset(ProgramCode , select = c("STUDENT_ID", "PROG_CODE", "CGPA"))
  ProgramCode$PROG_CODE <- sapply(ProgramCode$PROG_CODE , FUN = trimws)

  ProgramCode <- filter(ProgramCode, ProgramCode$PROG_CODE != "BS(AI)")
  ProgramCode <- filter(ProgramCode, ProgramCode$PROG_CODE != "BS(DS)")
  ProgramCode <- filter(ProgramCode, ProgramCode$PROG_CODE != "BS(CSDF)")
  ProgramCode <- filter(ProgramCode, ProgramCode$PROG_CODE != "BS(Math)")
  ProgramCode <- filter(ProgramCode, ProgramCode$PROG_CODE != "BS(SE)")
  ProgramCode <- filter(ProgramCode, ProgramCode$PROG_CODE != "BS(CE)")
  ProgramCode <- filter(ProgramCode, ProgramCode$PROG_CODE != "BS(TE)")

  ProgramCode$CGPA <- as.numeric(ProgramCode$CGPA)
  ProgramCode <- na.omit(ProgramCode)

  ProgramCode <- distinct(ProgramCode)

  Dataset <- inner_join(CourseData, ProgramCode)

}

## Data Cleaning

In [8]:
%%R
data_Cleaning1 <- function(student_data, semester_data){
        
    # Data Cleaning in Students Data File

    student_data$SCHOOL <- toupper(student_data$SCHOOL)
    student_data$COLLEGE <- toupper(student_data$COLLEGE)

    student_data$SCHOOL[student_data$SCHOOL == "-"] <- "others"
    student_data$COLLEGE[student_data$COLLEGE == "-"] <- "others"

    student_data$SCHOOL[student_data$SCHOOL %in% c("NOT KNOWN", "NOT GIVEN", "OTHERS")] <- "others"
    student_data$COLLEGE[student_data$COLLEGE %in% c("NOT KNOWN", "NOT GIVEN", "OTHERS")] <- "others"

    student_data$SCHOOL[nchar(student_data$SCHOOL) < 5] <- "others"
    student_data$COLLEGE[nchar(student_data$COLLEGE) < 5] <- "others"

    student_data$SCHOOL[is.na(student_data$SCHOOL)] <- "others"
    student_data$COLLEGE[is.na(student_data$COLLEGE)] <- "others"

    student_data$SCHOOL <- as.character(student_data$SCHOOL)
    student_data$COLLEGE <- as.character(student_data$COLLEGE)
    
    student_data$SEC_GRADE[is.na(student_data$SEC_GRADE)] <- round(mean(student_data$SEC_GRADE, na.rm=TRUE), digits=2)
    student_data$HIG_SEC_GRADE[is.na(student_data$HIG_SEC_GRADE)] <- round(mean(student_data$HIG_SEC_GRADE, na.rm=TRUE), digits=2)

    # Merging Semester data & student data

    training_data <- merge(semester_data, student_data, by = "STUDENT_ID")

    training_data <- training_data %>% filter(GENDER != "-")
    training_data$WARNING[is.na(training_data$WARNING)] <- 0

    training_data <- training_data %>% filter(!grepl("AI", training_data$PROG_CODE))
    training_data <- training_data %>% filter(!grepl("SE", training_data$PROG_CODE))
    training_data <- training_data %>% filter(!grepl("DS", training_data$PROG_CODE))
    training_data <- training_data %>% filter(!grepl("CSDF", training_data$PROG_CODE))

    training_data$SECONDARY <- as.character(training_data$SECONDARY)
    training_data$HIGHER_SECONDARY <- as.character(training_data$HIGHER_SECONDARY)

    training_data[is.na(training_data[, 42]), 42] <- "OTHER"
    training_data[is.na(training_data[, 45]), 45] <- "OTHER"

    training_data$SECONDARY <- as.factor(training_data$SECONDARY)
    training_data$HIGHER_SECONDARY <- as.factor(training_data$HIGHER_SECONDARY)

    testing_data <- training_data %>% filter(STATUS == "Current")
    training_data <- training_data %>% filter(STATUS == "Completed")

    for(i in c(2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30)){
      for (j in 1:nrow(training_data)){
        if (training_data[j, 32] >= i/2 && is.na(training_data[j, i])){
          training_data[j, i] = round(mean(training_data[, i], na.rm = TRUE), digits = 2)
        }
      }
    }

    training_data[c(2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30)][is.na(training_data[c(2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30)])] <- 0

    for (i in 1:nrow(training_data)){
      
      if(is.na(training_data$SEM_1_CGPA[i]) && training_data[i, 32] >= 1)
        training_data$SEM_1_CGPA[i] = round(training_data$SEM_1_SGPA[i], digits = 2)
      
      if(is.na(training_data$SEM_2_CGPA[i]) && training_data[i, 32] >= 2)
        training_data$SEM_2_CGPA[i] = round((training_data$SEM_1_SGPA[i] + training_data$SEM_2_SGPA[i])/2, digits = 2)
      
      if(is.na(training_data$SEM_3_CGPA[i]) && training_data[i, 32] >= 3)
        training_data$SEM_3_CGPA[i] = round((training_data$SEM_1_SGPA[i] + training_data$SEM_2_SGPA[i] + training_data$SEM_3_SGPA[i])/3, digits = 2)
      
      if(is.na(training_data$SEM_4_CGPA[i]) && training_data[i, 32] >= 4)
        training_data$SEM_4_CGPA[i] = round((training_data$SEM_1_SGPA[i] + training_data$SEM_2_SGPA[i] + training_data$SEM_3_SGPA[i] +
                                              training_data$SEM_4_SGPA[i])/4, digits = 2)
      
      if(is.na(training_data$SEM_5_CGPA[i]) && training_data[i, 32] >= 5)
        training_data$SEM_5_CGPA[i] = round((training_data$SEM_1_SGPA[i] + training_data$SEM_2_SGPA[i] + training_data$SEM_3_SGPA[i] +
                                              training_data$SEM_4_SGPA[i] + training_data$SEM_5_SGPA[i])/5, digits = 2)  
      
      if(is.na(training_data$SEM_6_CGPA[i]) && training_data[i, 32] >= 6)
        training_data$SEM_6_CGPA[i] = round((training_data$SEM_1_SGPA[i] + training_data$SEM_2_SGPA[i] + training_data$SEM_3_SGPA[i] +
                                              training_data$SEM_4_SGPA[i] + training_data$SEM_5_SGPA[i] + training_data$SEM_6_SGPA[i])/6,
                                            digits = 2)  
      
      if(is.na(training_data$SEM_7_CGPA[i]) && training_data[i, 32] >= 7)
        training_data$SEM_7_CGPA[i] = round((training_data$SEM_1_SGPA[i] + training_data$SEM_2_SGPA[i] + training_data$SEM_3_SGPA[i] +
                                              training_data$SEM_4_SGPA[i] + training_data$SEM_5_SGPA[i] + training_data$SEM_6_SGPA[i] +
                                              training_data$SEM_7_SGPA[i])/7, digits = 2)  
      
      if(is.na(training_data$SEM_8_CGPA[i]) && training_data[i, 32] >= 8)
        training_data$SEM_8_CGPA[i] = round((training_data$SEM_1_SGPA[i] + training_data$SEM_2_SGPA[i] +training_data$SEM_3_SGPA[i] +
                                              training_data$SEM_4_SGPA[i] + training_data$SEM_5_SGPA[i] + training_data$SEM_6_SGPA[i] +
                                              training_data$SEM_7_SGPA[i] + training_data$SEM_8_SGPA[i])/8, digits = 2)  
      
      if(is.na(training_data$SEM_9_CGPA[i]) && training_data[i, 32] >= 9)
        training_data$SEM_9_CGPA[i] = round((training_data$SEM_1_SGPA[i] + training_data$SEM_2_SGPA[i] + training_data$SEM_3_SGPA[i] +
                                              training_data$SEM_4_SGPA[i] + training_data$SEM_5_SGPA[i] + training_data$SEM_6_SGPA[i] +
                                              training_data$SEM_7_SGPA[i] + training_data$SEM_8_SGPA[i] + training_data$SEM_9_SGPA[i])/9,
                                            digits = 2)  
      
      if(is.na(training_data$SEM_10_CGPA[i]) && training_data[i, 32] >= 10)
        training_data$SEM_10_CGPA[i] = round((training_data$SEM_1_SGPA[i] + training_data$SEM_2_SGPA[i] + training_data$SEM_3_SGPA[i] +
                                                training_data$SEM_4_SGPA[i] + training_data$SEM_5_SGPA[i] + training_data$SEM_6_SGPA[i] +
                                                training_data$SEM_7_SGPA[i] + training_data$SEM_8_SGPA[i] + training_data$SEM_9_SGPA[i] +
                                                training_data$SEM_10_SGPA[i])/10, digits = 2) 
      
      if(is.na(training_data$SEM_11_CGPA[i]) && training_data[i, 32] >= 11)
        training_data$SEM_11_CGPA[i] = round((training_data$SEM_1_SGPA[i] + training_data$SEM_2_SGPA[i] + training_data$SEM_3_SGPA[i] +
                                                training_data$SEM_4_SGPA[i] + training_data$SEM_5_SGPA[i] + training_data$SEM_6_SGPA[i] +
                                                training_data$SEM_7_SGPA[i] + training_data$SEM_8_SGPA[i] + training_data$SEM_9_SGPA[i] +
                                                training_data$SEM_10_SGPA[i] + training_data$SEM_11_SGPA[i])/11, digits = 2) 
      
      if(is.na(training_data$SEM_12_CGPA[i]) && training_data[i, 32] >= 12)
        training_data$SEM_12_CGPA[i] = round((training_data$SEM_1_SGPA[i] + training_data$SEM_2_SGPA[i] + training_data$SEM_3_SGPA[i] +
                                                training_data$SEM_4_SGPA[i] + training_data$SEM_5_SGPA[i] + training_data$SEM_6_SGPA[i] +
                                                training_data$SEM_7_SGPA[i] + training_data$SEM_8_SGPA[i] + training_data$SEM_9_SGPA[i] +
                                                training_data$SEM_10_SGPA[i] + training_data$SEM_11_SGPA[i] + training_data$SEM_12_SGPA[i])/12,
                                            digits = 2) 
      
      if(is.na(training_data$SEM_13_CGPA[i]) && training_data[i, 32] >= 13)
        training_data$SEM_11_CGPA[i] = round((training_data$SEM_1_SGPA[i] + training_data$SEM_2_SGPA[i] + training_data$SEM_3_SGPA[i] +
                                                training_data$SEM_4_SGPA[i] + training_data$SEM_5_SGPA[i] + training_data$SEM_6_SGPA[i] +
                                                training_data$SEM_7_SGPA[i] + training_data$SEM_8_SGPA[i] +
                                                training_data$SEM_9_SGPA[i] + training_data$SEM_10_SGPA[i] + training_data$SEM_11_SGPA[i] +
                                                training_data$SEM_12_SGPA[i] + training_data$SEM_13_SGPA[i])/13, digits = 2) 
      
      if(is.na(training_data$SEM_14_CGPA[i]) && training_data[i, 32] >= 14)
        training_data$SEM_14_CGPA[i] = round((training_data$SEM_1_SGPA[i] + training_data$SEM_2_SGPA[i] + training_data$SEM_3_SGPA[i] +
                                                training_data$SEM_4_SGPA[i] + training_data$SEM_5_SGPA[i] + training_data$SEM_6_SGPA[i] +
                                                training_data$SEM_7_SGPA[i] + training_data$SEM_8_SGPA[i] + training_data$SEM_9_SGPA[i] +
                                                training_data$SEM_10_SGPA[i] + training_data$SEM_11_SGPA[i] + training_data$SEM_12_SGPA[i] +
                                                training_data$SEM_13_SGPA[i] + training_data$SEM_14_SGPA[i])/14, digits = 2) 
      
      if(is.na(training_data$SEM_15_CGPA[i]) && training_data[i, 32] >= 15)
        training_data$SEM_15_CGPA[i] = round((training_data$SEM_1_SGPA[i] + training_data$SEM_2_SGPA[i] + training_data$SEM_3_SGPA[i] +
                                                training_data$SEM_4_SGPA[i] + training_data$SEM_5_SGPA[i] + training_data$SEM_6_SGPA[i] +
                                                training_data$SEM_7_SGPA[i] + training_data$SEM_8_SGPA[i] + training_data$SEM_9_SGPA[i] +
                                                training_data$SEM_10_SGPA[i] + training_data$SEM_11_SGPA[i] + training_data$SEM_12_SGPA[i] +
                                                training_data$SEM_13_SGPA[i] + training_data$SEM_14_SGPA[i] + training_data$SEM_15_SGPA[i])/15,
                                            digits = 2)
    }

    training_data <- training_data %>% filter(TOTAL_SEM > 7 & TOTAL_SEM < 16)

    List <- c("SEM_1_SGPA", "SEM_2_SGPA", "SEM_3_SGPA", "SEM_4_SGPA", "SEM_5_SGPA", "SEM_6_SGPA", "SEM_7_SGPA", 
              "SEM_8_SGPA", "SEM_9_SGPA", "SEM_10_SGPA", "SEM_11_SGPA", "SEM_12_SGPA", "SEM_13_SGPA", "SEM_14_SGPA", 
              "SEM_15_SGPA", "WARNING", "STATUS")

    training_data <- training_data[, !(names(training_data) %in% List)]

    training_data[c(10, 11, 12, 13, 14, 15, 16)][is.na(training_data[c(10, 11, 12, 13, 14, 15, 16)])] <- 0

    testing_data <- testing_data %>% filter(TOTAL_SEM < 15)

    for (i in 1:nrow(testing_data)){
      j <- testing_data$TOTAL_SEM[i] * 2
      if(is.na(testing_data[i, j]) & !is.na(testing_data[i, j+1])){
        testing_data$TOTAL_SEM[i] <- testing_data$TOTAL_SEM[i] - 1
        testing_data[i, j+1] <- NA
      }
    }

    testing_data <- testing_data[, !(names(testing_data) %in% List)]

    data = list()
    data$training_data <- training_data
    data$testing_data <- testing_data
    
    return (data)
}

In [9]:
def data_Cleaning2(data, column, to_replace):
    
    ps = PorterStemmer()
    
    data[column] = data[column].str.lower()
    data[column] = data[column].str.strip()
    data[column] = data[column].str.replace('[-./()#?*@\d,:]+', '')
    data.loc[data[column] == "", column] = 'others'
    data.loc[data[column] == "na", column] = 'others'
    data.loc[data[column] == "not given", column] = 'others'
    data.loc[data[column] == "not known", column] = 'others'
    data[column] = data[column].str.strip()
    
    preprocessed_list = []
    for i in data[column]:
        list_ = i.split()
        stemmed = []
        word = " "
        for j in list_:
            stemmed.append(ps.stem(j))
        preprocessed_list.append(word.join(stemmed))
        
    data[column] = preprocessed_list
    
    for i in to_replace:
        strings = data[column].unique()
        matches = fuzzywuzzy.process.extract(i, strings, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
        close_matches = [matches[0] for matches in matches if matches[1] >= 55]
        rows_with_matches = data[column].isin(close_matches)
        data.loc[rows_with_matches, column] = i
        
    return data

## Model Building

In [10]:
def data_preprocessing(data):

    original = data
    
    ## Encoding data
    labelencoder = LabelEncoder()

    labelencoder.fit(data['GENDER'])
    data['GENDER'] = labelencoder.transform(data['GENDER'])
    gender_class = list(labelencoder.classes_)

    labelencoder.fit(data['BATCH'])
    data['BATCH'] = labelencoder.transform(data['BATCH'])
    batch_class = list(labelencoder.classes_)

    labelencoder.fit(data['CAMPUS'])
    data['CAMPUS'] = labelencoder.transform(data['CAMPUS'])
    campus_class = list(labelencoder.classes_)

    labelencoder.fit(data['PROG_CODE'])
    data['PROG_CODE'] = labelencoder.transform(data['PROG_CODE'])
    prog_code_class = list(labelencoder.classes_)

    labelencoder.fit(data['FIRST_SEM'])
    data['FIRST_SEM'] = labelencoder.transform(data['FIRST_SEM'])
    first_sem_class = list(labelencoder.classes_)

    labelencoder.fit(data['LAST_SEM'])
    data['LAST_SEM'] = labelencoder.transform(data['LAST_SEM'])
    last_sem_class = list(labelencoder.classes_)

    labelencoder.fit(data['CITY'])
    data['CITY'] = labelencoder.transform(data['CITY'])
    city_class = list(labelencoder.classes_)

    labelencoder.fit(data['SECONDARY'])
    data['SECONDARY'] = labelencoder.transform(data['SECONDARY'])
    sec_class = list(labelencoder.classes_)

    labelencoder.fit(data['HIGHER_SECONDARY'])
    data['HIGHER_SECONDARY'] = labelencoder.transform(data['HIGHER_SECONDARY'])
    hig_sec_class = list(labelencoder.classes_)

    labelencoder.fit(data['SCHOOL'])
    data['SCHOOL'] = labelencoder.transform(data['SCHOOL'])
    school_class = list(labelencoder.classes_)

    labelencoder.fit(data['COLLEGE'])
    data['COLLEGE'] = labelencoder.transform(data['COLLEGE'])
    college_class = list(labelencoder.classes_)
    
    ## Rearranging columns
    data = data[['SEM_1_CGPA', 'SEM_2_CGPA', 'SEM_3_CGPA', 'SEM_4_CGPA', 'SEM_5_CGPA', 'SEM_6_CGPA', 'SEM_7_CGPA', 'TOTAL_SEM', 
                 'GENDER', 'BATCH', 'CAMPUS', 'PROG_CODE', 'FIRST_SEM', 'LAST_SEM', 'CITY', 'SECONDARY', 'SCHOOL', 'SEC_GRADE', 
                 'HIGHER_SECONDARY', 'COLLEGE', 'HIG_SEC_GRADE', 'CGPA']]
    
    return data

In [11]:
def linear_regression(training_data):
    train = pd.DataFrame()
    train = data_preprocessing(training_data)

    X = train.iloc[:, :-1]
    y = train.iloc[:, -1]
    X = (X - X.min()) / (X.max() - X.min())
    X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=1)

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_validate)
    model_acc = r2_score(y_pred, y_validate)
    rmse_ = np.sqrt(mean_squared_error(y_validate, y_pred))
    
    return model_acc

In [12]:
def gnb(training_data):

    train = pd.DataFrame()
    train = data_preprocessing(training_data)
    
    labelencoder = LabelEncoder()

    X = train.iloc[:, :-1]
    X = (X - X.min()) / (X.max() - X.min())

    y = labelencoder.fit_transform(pd.cut(train['CGPA'], 5, retbins=True)[0])

    X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.3, random_state=1)

    model = GaussianNB()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_validate)
    model_acc = accuracy_score(y_validate, y_pred)
    
    return model_acc

In [13]:
def logistic_regression(training_data):

    train = pd.DataFrame()
    train = data_preprocessing(training_data)
    labelencoder = LabelEncoder()

    X = train.iloc[:, :-1]
    X = (X - X.min()) / (X.max() - X.min())

    y = labelencoder.fit_transform(pd.cut(train['CGPA'], 5, retbins=True)[0])

    X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.3, random_state=1)

    model = LogisticRegression(solver='sag', multi_class='multinomial')
    model.fit(X_train, y_train)
    model_acc = model.score(X_validate, y_validate)
    
    return model_acc

In [14]:
def knn(training_data):
 
    train = pd.DataFrame()
    train = data_preprocessing(training_data)
    
    labelencoder = LabelEncoder()

    X = train.iloc[:, :-1]
    X = (X - X.min()) / (X.max() - X.min())

    y = labelencoder.fit_transform(pd.cut(train['CGPA'], 5, retbins=True)[0])

    X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.3, random_state=1)

    model = KNeighborsClassifier(n_neighbors=3) 
    model.fit(X_train, y_train) 
    y_pred = model.predict(X_validate) 
    model_acc = accuracy_score(y_validate, y_pred) 
    
    return model_acc

In [15]:
def decisiontree(training_data):
 
    train = pd.DataFrame()
    train = data_preprocessing(training_data)

    labelencoder = LabelEncoder()

    X = train.iloc[:, :-1]
    X = (X - X.min()) / (X.max() - X.min())

    y = labelencoder.fit_transform(pd.cut(train['CGPA'], 5, retbins=True)[0])

    X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.3, random_state=1)

    model = DecisionTreeClassifier(criterion = 'entropy', ccp_alpha = 0.015)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_validate)
    model_acc = accuracy_score(y_validate, y_pred)
    
    return model_acc

In [16]:
def randomforest(training_data):
    
    train = pd.DataFrame()
    train = data_preprocessing(training_data)
    
    labelencoder = LabelEncoder()

    X = train.iloc[:, :-1]
    X = (X - X.min()) / (X.max() - X.min())

    y = labelencoder.fit_transform(pd.cut(train['CGPA'], 5, retbins=True)[0])

    X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.3, random_state=1)

    model = RandomForestClassifier(n_estimators=100, bootstrap = True, max_features = 'sqrt', criterion = 'entropy')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_validate)
    model_acc = accuracy_score(y_validate, y_pred)
    
    return model_acc

In [17]:
def svm(training_data):
    
    train = pd.DataFrame()
    train = data_preprocessing(training_data)
    
    labelencoder = LabelEncoder()

    X = train.iloc[:, :-1]
    X = (X - X.min()) / (X.max() - X.min())

    y = labelencoder.fit_transform(pd.cut(train['CGPA'], 5, retbins=True)[0])

    X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.3, random_state=1)

    model = SVC(kernel="linear", C=1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_validate)
    model_acc = accuracy_score(y_validate, y_pred)
    
    return model_acc

## Predictive Modeling

In [18]:
def predict_1st_semester(training_data, testing_data):
    
    train = pd.DataFrame()
    train = data_preprocessing(training_data)

    test = pd.DataFrame()
    test = data_preprocessing(testing_data)
    test = test.fillna(0)

    X = train.iloc[:, :-1]
    y = train.iloc[:, -1]
    X_test = test.iloc[:, :-1]

    X_ = X[[ 'TOTAL_SEM', 'GENDER', 'BATCH', 'CAMPUS','PROG_CODE', 'FIRST_SEM', 'LAST_SEM', 'CITY', 'SECONDARY', 'SCHOOL',
              'SEC_GRADE', 'HIGHER_SECONDARY', 'COLLEGE', 'HIG_SEC_GRADE']]

    test_sem_1 = X_test[X_test["TOTAL_SEM"] == 0]
    test_sem_1 = test_sem_1[['TOTAL_SEM', 'GENDER', 'BATCH', 'CAMPUS','PROG_CODE', 'FIRST_SEM', 'LAST_SEM', 'CITY', 
                              'SECONDARY', 'SCHOOL','SEC_GRADE', 'HIGHER_SECONDARY', 'COLLEGE', 'HIG_SEC_GRADE']]

    print("Predict CGPA of 1st Semester\nTotal Records:", len(test_sem_1))

    model = LinearRegression()
    model.fit(X_, y)
    y_pred = np.round(model.predict(test_sem_1), 2)
    test_sem_1['SEM_1_CGPA'] = y_pred

    return test_sem_1

In [19]:
def predict_2nd_semester(training_data, testing_data):
    
    train = pd.DataFrame()
    train = data_preprocessing(training_data)

    test = pd.DataFrame()
    test = data_preprocessing(testing_data)
    test = test.fillna(0)

    X = train.iloc[:, :-1]
    y = train.iloc[:, -1]
    X_test = test.iloc[:, :-1]

    X_ = X[['SEM_1_CGPA', 'TOTAL_SEM', 'GENDER', 'BATCH', 'CAMPUS','PROG_CODE', 'FIRST_SEM', 'LAST_SEM', 'CITY', 'SECONDARY', 'SCHOOL',
              'SEC_GRADE', 'HIGHER_SECONDARY', 'COLLEGE', 'HIG_SEC_GRADE']]

    test_sem_2 = X_test[X_test["TOTAL_SEM"] == 1]
    test_sem_2 = test_sem_2[['SEM_1_CGPA', 'TOTAL_SEM', 'GENDER', 'BATCH', 'CAMPUS','PROG_CODE', 'FIRST_SEM', 'LAST_SEM', 'CITY', 
                              'SECONDARY', 'SCHOOL','SEC_GRADE', 'HIGHER_SECONDARY', 'COLLEGE', 'HIG_SEC_GRADE']]

    print("Predict CGPA of 2nd Semester\nTotal Records:", len(test_sem_2))

    model = LinearRegression()
    model.fit(X_, y)
    y_pred = np.round(model.predict(test_sem_2), 2)
    test_sem_2['SEM_2_CGPA'] = y_pred

    return test_sem_2

In [20]:
def predict_3rd_semester(training_data, testing_data):

    train = pd.DataFrame()
    train = data_preprocessing(training_data)

    test = pd.DataFrame()
    test = data_preprocessing(testing_data)
    test = test.fillna(0)
    
    X = train.iloc[:, :-1]
    y = train.iloc[:, -1]
    X_test = test.iloc[:, :-1]

    X_ = X[['SEM_1_CGPA', 'SEM_2_CGPA', 'TOTAL_SEM', 'GENDER', 'BATCH', 'CAMPUS','PROG_CODE', 'FIRST_SEM', 'LAST_SEM', 'CITY', 
              'SECONDARY', 'SCHOOL', 'SEC_GRADE', 'HIGHER_SECONDARY', 'COLLEGE', 'HIG_SEC_GRADE']]

    test_sem_3 = X_test[X_test["TOTAL_SEM"] == 2]
    test_sem_3 = test_sem_3[['SEM_1_CGPA', 'SEM_2_CGPA', 'TOTAL_SEM', 'GENDER', 'BATCH', 'CAMPUS','PROG_CODE', 'FIRST_SEM', 
                              'LAST_SEM', 'CITY', 'SECONDARY', 'SCHOOL','SEC_GRADE', 'HIGHER_SECONDARY', 'COLLEGE', 'HIG_SEC_GRADE']]

    print("\nPredict CGPA of 3rd Semester\nTotal Records:", len(test_sem_3))

    model = LinearRegression()
    model.fit(X_, y)
    y_pred = np.round(model.predict(test_sem_3), 2)
    test_sem_3['SEM_3_CGPA'] = y_pred

    return test_sem_3

In [21]:
def predict_4th_semester(training_data, testing_data):
    
    train = pd.DataFrame()
    train = data_preprocessing(training_data)

    test = pd.DataFrame()
    test = data_preprocessing(testing_data)
    test = test.fillna(0)

    X = train.iloc[:, :-1]
    y = train.iloc[:, -1]
    X_test = test.iloc[:, :-1]

    X_ = X[['SEM_1_CGPA', 'SEM_2_CGPA', 'SEM_3_CGPA', 'TOTAL_SEM', 'GENDER', 'BATCH', 'CAMPUS','PROG_CODE', 'FIRST_SEM', 
              'LAST_SEM', 'CITY', 'SECONDARY', 'SCHOOL', 'SEC_GRADE', 'HIGHER_SECONDARY', 'COLLEGE', 'HIG_SEC_GRADE']]

    test_sem_4 = X_test[X_test["TOTAL_SEM"] == 3]
    test_sem_4 = test_sem_4[['SEM_1_CGPA', 'SEM_2_CGPA', 'SEM_3_CGPA', 'TOTAL_SEM', 'GENDER', 'BATCH', 'CAMPUS','PROG_CODE', 
                              'FIRST_SEM', 'LAST_SEM', 'CITY', 'SECONDARY', 'SCHOOL','SEC_GRADE', 'HIGHER_SECONDARY', 'COLLEGE', 
                              'HIG_SEC_GRADE']]

    print("\nPredict CGPA of 4th Semester\nTotal Records:", len(test_sem_4))

    model = LinearRegression()
    model.fit(X_, y)
    y_pred = np.round(model.predict(test_sem_4), 2)
    test_sem_4['SEM_4_CGPA'] = y_pred
    
    return test_sem_4

In [22]:
def predict_5th_semester(training_data, testing_data):
    
    train = pd.DataFrame()
    train = data_preprocessing(training_data)

    test = pd.DataFrame()
    test = data_preprocessing(testing_data)
    test = test.fillna(0)

    X = train.iloc[:, :-1]
    y = train.iloc[:, -1]
    X_test = test.iloc[:, :-1]

    X_ = X[['SEM_1_CGPA', 'SEM_2_CGPA', 'SEM_3_CGPA', 'SEM_4_CGPA', 'TOTAL_SEM', 'GENDER', 'BATCH', 'CAMPUS','PROG_CODE', 
        'FIRST_SEM', 'LAST_SEM', 'CITY', 'SECONDARY', 'SCHOOL', 'SEC_GRADE', 'HIGHER_SECONDARY', 'COLLEGE', 'HIG_SEC_GRADE']]

    test_sem_5 = X_test[X_test["TOTAL_SEM"] == 4]
    test_sem_5 = test_sem_5[['SEM_1_CGPA', 'SEM_2_CGPA', 'SEM_3_CGPA', 'SEM_4_CGPA', 'TOTAL_SEM', 'GENDER', 'BATCH', 'CAMPUS',
                          'PROG_CODE', 'FIRST_SEM', 'LAST_SEM', 'CITY', 'SECONDARY', 'SCHOOL','SEC_GRADE', 'HIGHER_SECONDARY', 
                          'COLLEGE', 'HIG_SEC_GRADE']]

    print("\nPredict CGPA of 5th Semester\nTotal Records:", len(test_sem_5))

    model = LinearRegression()
    model.fit(X_, y)
    y_pred = np.round(model.predict(test_sem_5), 2)
    test_sem_5['SEM_5_CGPA'] = y_pred
    
    return test_sem_5

In [23]:
def predict_6th_semester(training_data, testing_data):
    
    train = pd.DataFrame()
    train = data_preprocessing(training_data)

    test = pd.DataFrame()
    test = data_preprocessing(testing_data)
    test = test.fillna(0)

    X = train.iloc[:, :-1]
    y = train.iloc[:, -1]
    X_test = test.iloc[:, :-1]

    X_ = X[['SEM_1_CGPA', 'SEM_2_CGPA', 'SEM_3_CGPA', 'SEM_4_CGPA', 'SEM_5_CGPA', 'TOTAL_SEM', 'GENDER', 'BATCH', 'CAMPUS',
        'PROG_CODE', 'FIRST_SEM', 'LAST_SEM', 'CITY', 'SECONDARY', 'SCHOOL', 'SEC_GRADE', 'HIGHER_SECONDARY', 'COLLEGE', 
        'HIG_SEC_GRADE']]

    test_sem_6 = X_test[X_test["TOTAL_SEM"] == 5]
    test_sem_6 = test_sem_6[['SEM_1_CGPA', 'SEM_2_CGPA', 'SEM_3_CGPA', 'SEM_4_CGPA', 'SEM_5_CGPA', 'TOTAL_SEM', 'GENDER', 'BATCH', 
                          'CAMPUS','PROG_CODE', 'FIRST_SEM', 'LAST_SEM', 'CITY', 'SECONDARY', 'SCHOOL','SEC_GRADE', 
                          'HIGHER_SECONDARY', 'COLLEGE', 'HIG_SEC_GRADE']]

    print("\nPredict CGPA of 6th Semester\nTotal Records:", len(test_sem_6))

    model = LinearRegression() 
    model.fit(X_, y)
    y_pred = np.round(model.predict(test_sem_6), 2)
    test_sem_6['SEM_6_CGPA'] = y_pred
    
    return test_sem_6

In [24]:
def predict_7th_semester(training_data, testing_data):

    train = pd.DataFrame()
    train = data_preprocessing(training_data)

    test = pd.DataFrame()
    test = data_preprocessing(testing_data)
    test = test.fillna(0)
    
    X = train.iloc[:, :-1]
    y = train.iloc[:, -1]
    X_test = test.iloc[:, :-1]

    X_ = X[['SEM_1_CGPA', 'SEM_2_CGPA', 'SEM_3_CGPA', 'SEM_4_CGPA', 'SEM_5_CGPA', 'SEM_6_CGPA', 'TOTAL_SEM', 'GENDER', 'BATCH', 
              'CAMPUS', 'PROG_CODE', 'FIRST_SEM', 'LAST_SEM', 'CITY', 'SECONDARY', 'SCHOOL', 'SEC_GRADE', 'HIGHER_SECONDARY', 
              'COLLEGE', 'HIG_SEC_GRADE']]

    test_sem_7 = X_test[X_test["TOTAL_SEM"] == 6]
    test_sem_7 = test_sem_7[['SEM_1_CGPA', 'SEM_2_CGPA', 'SEM_3_CGPA', 'SEM_4_CGPA', 'SEM_5_CGPA', 'SEM_6_CGPA', 'TOTAL_SEM', 
                              'GENDER', 'BATCH', 'CAMPUS','PROG_CODE', 'FIRST_SEM', 'LAST_SEM', 'CITY', 'SECONDARY', 'SCHOOL',
                              'SEC_GRADE', 'HIGHER_SECONDARY', 'COLLEGE', 'HIG_SEC_GRADE']]

    print("\nPredict CGPA of 7th Semester\nTotal Records:", len(test_sem_7))

    model = LinearRegression()
    model.fit(X_, y)
    y_pred = np.round(model.predict(test_sem_7), 2)
    test_sem_7['SEM_7_CGPA'] = y_pred
    
    return test_sem_7

In [25]:
def predict_8th_semester(training_data, testing_data):

    train = pd.DataFrame()
    train = data_preprocessing(training_data)

    test = pd.DataFrame()
    test = data_preprocessing(testing_data)
    test = test.fillna(0)
    
    X = train.iloc[:, :-1]
    y = train.iloc[:, -1]
    X_test = test.iloc[:, :-1]

    X_ = X[['SEM_1_CGPA', 'SEM_2_CGPA', 'SEM_3_CGPA', 'SEM_4_CGPA', 'SEM_5_CGPA', 'SEM_6_CGPA', 'SEM_7_CGPA', 'TOTAL_SEM', 'GENDER', 
            'BATCH', 'CAMPUS', 'PROG_CODE', 'FIRST_SEM', 'LAST_SEM', 'CITY', 'SECONDARY', 'SCHOOL', 'SEC_GRADE', 'HIGHER_SECONDARY', 
            'COLLEGE', 'HIG_SEC_GRADE']]

    test_sem_8 = X_test[X_test["TOTAL_SEM"] == 7]
    test_sem_8 = test_sem_8[['SEM_1_CGPA', 'SEM_2_CGPA', 'SEM_3_CGPA', 'SEM_4_CGPA', 'SEM_5_CGPA', 'SEM_6_CGPA', 'SEM_7_CGPA', 
                              'TOTAL_SEM', 'GENDER', 'BATCH', 'CAMPUS','PROG_CODE', 'FIRST_SEM', 'LAST_SEM', 'CITY', 'SECONDARY', 
                              'SCHOOL','SEC_GRADE', 'HIGHER_SECONDARY', 'COLLEGE', 'HIG_SEC_GRADE']]

    print("\nPredict CGPA of 8th Semester\nTotal Records:", len(test_sem_8))

    model = LinearRegression()
    model.fit(X_, y)
    y_pred = np.round(model.predict(test_sem_8), 2)
    test_sem_8['SEM_8_CGPA'] = y_pred
    
    return test_sem_8

In [26]:
%R -o read_data
%R -o student_Preprocessing
%R -o semester_Preprocessing
%R -o course_Preprocessing
%R -o data_Cleaning1

In [394]:
class Predictive_Analytics:
    
    def __init__(self, root):
        
        self.root = root
        self.root.geometry('1350x700+0+0')
        self.root.title('Predictive Analytics on Academic Record of NUCES')

        title = Label(self.root, text='Predictive Analytics on the Academic Record of NUCES', bd=12, relief=GROOVE, bg='#152238', fg='#98FB98', font=('times new roman', 25, 'bold'), pady=2).pack(fill=X)

        #-----------------------------------------------------------------------------

        self.CourseData1 = pd.DataFrame()
        self.CourseData2 = pd.DataFrame()
        self.CourseData = pd.DataFrame()
        self.st_data = pd.DataFrame()
        
        self.StudentData = pd.DataFrame()
        self.SemesterData = pd.DataFrame()
        
        self.training_data = pd.DataFrame()
        self.testing_data = pd.DataFrame()
        
        self.test_sem = pd.DataFrame()
        
        self.areaFrame1 = False
        self.areaFrame3 = False
        self.studentDataImport = False
        self.courseDataImport = False
        self.studentDataTransform = False
        self.courseDataTransform = False
        self.studentDataClean = False
        self.courseDataClean = False
        self._canvas__ = False
        
        school = open("Data/Schools.txt", "r")
        self.unique_schools = school.read().splitlines()
        school.close()

        college = open("Data/Colleges.txt", "r")
        self.unique_colleges = college.read().splitlines()
        college.close()

        self.accuracy = StringVar()
        
        #-----------------------------------------------------------------------------

        menubar = Menu(self.root)  

        Data_Transformation = Menu(menubar, tearoff=0)  
        Data_Transformation.add_command(label="Student Data", command=self.transform_student_data)  
        Data_Transformation.add_command(label="Course Data", command=self.transform_course_data) 
        menubar.add_cascade(label="Data Transformation", menu=Data_Transformation) 

        #---------------------------------------------------------

        Data_Cleaning = Menu(menubar, tearoff=0)  
        Data_Cleaning.add_command(label="Student Data", command=self.clean_student_data)   
        Data_Cleaning.add_command(label="Course Data", command=self.not_defined) 
        menubar.add_cascade(label="Data Cleaning", menu=Data_Cleaning) 

        #---------------------------------------------------------

        Visualization = Menu(menubar, tearoff=0) 

        submenu = Menu(Visualization, tearoff=0)
        submenu.add_command(label="Students In each Campus", command=self.students_in_each_campus)
        submenu.add_command(label="Students In each Degree Program", command=self.students_in_each_program)
        submenu.add_command(label="Programs Offered In each Campus", command=self.program_at_each_campus)
        submenu.add_command(label="Total Male and Female Students", command=self.gender_of_students)
        submenu.add_command(label="Total Male and Female Students In each Campus", command=self.gender_of_students_at_each_campus)
        submenu.add_command(label="Secondary Education Background", command=self.secondary_background)
        submenu.add_command(label="Higher Secondary Education Background", command=self.hig_secondary_background)
        submenu.add_command(label="Performance In each Campus", command=self.performance_at_each_campus)
        submenu.add_command(label="BoxPlot of CS Degree", command=self.not_defined)
        submenu.add_command(label="BoxPlot of EE Degree", command=self.not_defined)
        submenu.add_command(label="BoxPlot of BBA Degree", command=self.not_defined)
        submenu.add_command(label="Summary of CS Degree", command=self.not_defined)
        submenu.add_command(label="Summary of EE Degree", command=self.not_defined)
        submenu.add_command(label="Summary of BBA Degree", command=self.not_defined)
        Visualization.add_cascade(label='Whole Data', menu=submenu)

        submenu = Menu(Visualization, tearoff=0)
        submenu.add_command(label="Students In each Degree Program", command=lambda:self.students_in_each_program("FSD"))
        submenu.add_command(label="Total Male and Female Students", command=lambda:self.gender_of_students("FSD"))
        submenu.add_command(label="Number of Students from different Cities", command=lambda:self.students_in_each_city("FSD"))
        submenu.add_command(label="Secondary Education Background", command=lambda:self.secondary_background("FSD"))
        submenu.add_command(label="Higher Secondary Education Background", command=lambda:self.hig_secondary_background("FSD"))
        Visualization.add_cascade(label='Faisalabad Data', menu=submenu)

        submenu = Menu(Visualization, tearoff=0)
        submenu.add_command(label="Students In each Degree Program", command=lambda:self.students_in_each_program("ISB"))
        submenu.add_command(label="Total Male and Female Students", command=lambda:self.gender_of_students("ISB"))
        submenu.add_command(label="Number of Students from different Cities", command=lambda:self.students_in_each_city("ISB"))
        submenu.add_command(label="Secondary Education Background", command=lambda:self.secondary_background("ISB"))
        submenu.add_command(label="Higher Secondary Education Background", command=lambda:self.hig_secondary_background("ISB"))
        Visualization.add_cascade(label='Islamabad Data', menu=submenu)

        submenu = Menu(Visualization, tearoff=0)
        submenu.add_command(label="Students In each Degree Program", command=lambda:self.students_in_each_program("KHI"))
        submenu.add_command(label="Total Male and Female Students", command=lambda:self.gender_of_students("KHI"))
        submenu.add_command(label="Number of Students from different Cities", command=lambda:self.students_in_each_city("KHI"))
        submenu.add_command(label="Secondary Education Background", command=lambda:self.secondary_background("KHI"))
        submenu.add_command(label="Higher Secondary Education Background", command=lambda:self.hig_secondary_background("KHI"))
        Visualization.add_cascade(label='Karachi Data', menu=submenu)

        submenu = Menu(Visualization, tearoff=0)
        submenu.add_command(label="Students In each Degree Program", command=lambda:self.students_in_each_program("LHR"))
        submenu.add_command(label="Total Male and Female Students", command=lambda:self.gender_of_students("LHR"))
        submenu.add_command(label="Number of Students from different Cities", command=lambda:self.students_in_each_city("LHR"))
        submenu.add_command(label="Secondary Education Background", command=lambda:self.secondary_background("LHR"))
        submenu.add_command(label="Higher Secondary Education Background", command=lambda:self.hig_secondary_background("LHR"))
        Visualization.add_cascade(label='Lahore Data', menu=submenu)

        submenu = Menu(Visualization, tearoff=0)
        submenu.add_command(label="Students In each Degree Program", command=lambda:self.students_in_each_program("PWR"))
        submenu.add_command(label="Total Male and Female Students", command=lambda:self.gender_of_students("PWR"))
        submenu.add_command(label="Number of Students from different Cities", command=lambda:self.students_in_each_city("PWR"))
        submenu.add_command(label="Secondary Education Background", command=lambda:self.secondary_background("PWR"))
        submenu.add_command(label="Higher Secondary Education Background", command=lambda:self.hig_secondary_background("PWR"))
        Visualization.add_cascade(label='Peshawar Data', menu=submenu)

        menubar.add_cascade(label="Visualization", menu=Visualization) 

        #---------------------------------------------------------

        Predictions = Menu(menubar, tearoff=0)  
        Predictions.add_command(label="Predict Semester 1", command=self.P_1st) 
        Predictions.add_command(label="Predict Semester 2", command=self.P_2nd) 
        Predictions.add_command(label="Predict Semester 3", command=self.P_3rd) 
        Predictions.add_command(label="Predict Semester 4", command=self.P_4th) 
        Predictions.add_command(label="Predict Semester 5", command=self.P_5th) 
        Predictions.add_command(label="Predict Semester 6", command=self.P_6th) 
        Predictions.add_command(label="Predict Semester 7", command=self.P_7th) 
        Predictions.add_command(label="Predict Semester 8", command=self.P_8th) 

        menubar.add_cascade(label="Predictive Modeling", menu=Predictions)

        self.root.config(menu=menubar) 

        #-------------------------------------------View Data Output Frame---------------------------------

        self.frame1 = Frame(self.root, bd=10, relief=GROOVE)
        self.frame1.place(x=0, y=75, width=445, height=420)

        f1_title = Label(self.frame1, text='Data', bd=12, relief=GROOVE, bg='#152238', fg='#98FB98', font=('times new roman', 14, 'bold'), pady=2).pack(fill=X)

        self.f1_scroll_x = Scrollbar(self.frame1, orient=HORIZONTAL)
        self.f1_scroll_x.pack(side=BOTTOM, fill=X)

        self.f1_scroll_y = Scrollbar(self.frame1, orient=VERTICAL)
        self.f1_scroll_y.pack(side=RIGHT, fill=Y)

        # ----------------------------------------Visualization Frame--------------------------------

        self.frame2 = LabelFrame(self.root, labelanchor='n', bd=12, relief=GROOVE, fg='#01796F', font=('times new roman', 15, 'bold'))
        self.frame2.place(x=455, y=75, width=450, height=420)

        f2_title = Label(self.frame2, text='Visualizations', bd=12, relief=GROOVE, bg='#152238', fg='light green', font=('times new roman', 14, 'bold'), pady=2).pack(fill=X)

        # ------------------------------------- Predictive Modeling Output Frame------------------------
        
        self.frame3 = Frame(root, bd=10, relief=GROOVE)
        self.frame3.place(x=900, y=75, width=455, height=420)

        f3_title = Label(self.frame3, text='Predictive Modeling', bd=12, relief=GROOVE, bg='#152238', fg='#98FB98', font=('times new roman', 15, 'bold'), pady=2).pack(fill=X)

        self.f3_scroll_x = Scrollbar(self.frame3, orient=HORIZONTAL)
        self.f3_scroll_x.pack(side=BOTTOM, fill=X)

        self.f3_scroll_y = Scrollbar(self.frame3, orient=VERTICAL)
        self.f3_scroll_y.pack(side=RIGHT, fill=Y)

        # --------------------------------------Frame for Model Building & GUI Buttons------------------------

        frame4 = LabelFrame(self.root, labelanchor='n', bd=12, relief=GROOVE, bg='#152238', fg='#01796F', font=('times new roman', 15, 'bold'))
        frame4.place(x=0, y=500, relwidth=1, height=200)  

        f4_itm1 = Frame(frame4, bd=7, relief=GROOVE)
        f4_itm1.place(x=0, y=2, width=770, height=172) 
        
        #--------------------------------------------Model Building Frame-------------------

        f4_title_itm1 = Label(f4_itm1, text='Model Building', bd=12, relief=GROOVE, bg='#152238', fg='lightgreen', font=('times new roman', 14, 'bold'), pady=2).pack(fill=X)
        
        lr_ = Button(frame4, text="Linear Regression", command=self.lr__, bg='#01796F', fg='white', bd=5, width=16, height=2, font='arial 9').place(x=8, y=62)
        nb_ = Button(frame4, text="Naive Bayes", command=self.gnb__, bg='#01796F', fg='white', bd=5, width=13, height=2, font='arial 9').place(x=142, y=62)
        logr_ = Button(frame4, text="Logistic Regression", command=self.logR__, bg='#01796F', fg='white', bd=5, width=16, height=2, font='arial 9').place(x=254, y=62)
        knn_ = Button(frame4, text="KNN", bg='#01796F', command=self.knn__, fg='white', bd=5, width=8, height=2, font='arial 9').place(x=388, y=62)
        dt_ = Button(frame4, text="Decision Tree",  command=self.dt__, bg='#01796F', fg='white', bd=5, width=12, height=2, font='arial 9').place(x=466, y=62)
        rf_ = Button(frame4, text="Random Forest", command=self.rf__, bg='#01796F', fg='white', bd=5, width=14, height=2, font='arial 9').place(x=570, y=62)
        svm_ = Button(frame4, text="SVM", command=self.svm__, bg='#01796F', fg='white', bd=5, width=8, height=2, font='arial 9').place(x=688, y=62)
        
        f4_itm2 = Frame(frame4, bd=7, relief=GROOVE, bg='#152238')
        f4_itm2.place(x=8, y=112, width=756, height=60)
        
        f4_itm2_acc = Label(f4_itm2, text='Accuracy', bg='#152238', fg='lightgreen',font=('times new roman', 18, 'bold')).place(x=190, y=8)
        f4_itm2_txt = Entry(f4_itm2, textvariable=self.accuracy, font=('times new roman', 16), bd=5, relief=SUNKEN, width=13).place(x=320, y=6)
        
        f4_itm3 = Frame(frame4, bd=7, relief=GROOVE)
        f4_itm3.place(x=780, y=1, width=540, height=72)
        
        #------------------------------------------GUI Buttons---------------------------------

        import_st_btn = Button(f4_itm3, text='Import Student Data', command=self.import_student_data, bg='#01796F', fg='white', bd=5, width=17, height=2, font='arial 9').grid(row=0, column=0, padx=7, pady=4)

        import_cr_btn = Button(f4_itm3, text='Import Course Data', command=self.import_course_data, bg='#01796F', fg='white', bd=5, width=17, height=2, font='arial 9').grid(row=0, column=1, padx=7, pady=4)

        clear_btn = Button(f4_itm3, text='Clear Data', command=self.clear_data, bg='#01796F', fg='white', bd=5, width=13, height=2, font='arial 9').grid(row=0, column=2, padx=7, pady=4)

        exit_btn = Button(f4_itm3, text='Exit', command=self.exit, bg='#01796F', fg='white', bd=5, width=10, height=2, font='arial 9').grid(row=0, column=3, padx=7, pady=4)

        f4_itm4 = LabelFrame(frame4, text='Select Data to View', labelanchor='n', bd=12, relief=SUNKEN, fg='#01796F', font=('times new roman', 10, 'bold'))
        f4_itm4.place(x=780, y=82, width=540, height=88)
                
        d1 = Button(f4_itm4, text='Student Data', command=self.view_student_data, bg='#01796F', fg='white', bd=5, width=20, height=1, font='arial 9').grid(row=0, column=0, padx=70, pady=12)
        
        d2 = Button(f4_itm4, text='Course Data', bg='#01796F', fg='white', bd=5, width=20, height=1, font='arial 9').grid(row=0, column=1, pady=12)

    #------------------------------------------------Function Definitions----------------------------------
        
    #--------------------------------------Import Functions------------------------
    
    def import_student_data(self):
        self.studentDataImport = True
        self.StudentData = read_data("Student_Data")
        self.SemesterData = read_data("Semester Data")
        self.st_data = self.StudentData
        messagebox.showinfo("Alert Message", "Dataset Uploaded!")
            
    def import_course_data(self):
        self.courseDataImport = True
        self.CourseData1 = read_data("Course Data - 1") 
        self.CourseData2 = read_data("Course Data - 2")
        messagebox.showinfo("Alert Message", "Dataset Uploaded!")
        
    #------------------------------------Transformation Functions--------------------
    
    def transform_student_data(self):
        if (self.studentDataImport):
            messagebox.showinfo("Message", "Transforming Data")
            self.StudentData = student_Preprocessing(self.StudentData)
            print("StudentData")
            self.SemesterData = semester_Preprocessing(self.SemesterData)
            print("SemesterData")
            messagebox.showinfo("Alert Message", "Student Data Transformed!")  
            self.studentDataTransform = True
        else:
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        
    def transform_course_data(self):
        if (self.courseDataImport):
            messagebox.showinfo("Message", "Transforming Data")
            self.CourseData = course_Preprocessing(self.CourseData1, self.CourseData2, self.st_data)
            print("CourseData")
            messagebox.showinfo("Alert Message", "Course Data Transformed!")
            self.courseDataTransform = True
        else:
            messagebox.showinfo("Error", "Please, Import 'Course Data'")
            
    #------------------------------------Data Cleaning Functions------------------------
            
    def clean_student_data(self):
        if (self.studentDataImport & self.studentDataTransform):
            messagebox.showinfo("Message", "Cleaning Data")
            data = data_Cleaning1(self.StudentData, self.SemesterData)
            self.training_data = data[data.names.index('training_data')]
            self.testing_data = data[data.names.index('testing_data')]
            with localconverter(ro.default_converter + pandas2ri.converter):
                self.training_data = ro.conversion.rpy2py(self.training_data)
                self.testing_data = ro.conversion.rpy2py(self.testing_data)
            self.training_data = data_Cleaning2(self.training_data, "SCHOOL", self.unique_schools)
            self.training_data = data_Cleaning2(self.training_data, "COLLEGE", self.unique_colleges)
            self.testing_data = data_Cleaning2(self.testing_data, "SCHOOL", self.unique_schools)
            self.testing_data = data_Cleaning2(self.testing_data, "COLLEGE", self.unique_colleges)
            self.studentDataClean = True
            print("Student Data Cleaned")
            messagebox.showinfo("Alert Message", "Student Data Cleaned")
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")

    def clean_course_data(self):
        messagebox.showinfo("Message", "Cleaning Data")
        messagebox.showinfo("Alert Message", "Course Data Cleaned")
        
        
    #--------------------------------Visualization Functions------------------------------------

    def gender_of_students_at_each_campus(self):
        figure = Figure(figsize=(10, 10))
        ax = figure.add_subplot(111)
        b = sns.countplot(x="CAMPUS", hue="GENDER", data = self.get_data(), ax=ax)                                                                                                           
        b.set(ylabel="No. of Students", xlabel="Gender of Students", title="Female & Male Students at Each Campus")
        self.plot(figure)
        
    def students_in_each_campus(self):
        figure = Figure(figsize=(10, 10))
        ax = figure.add_subplot(111)
        b = sns.countplot(x="CAMPUS", data = self.get_data(), ax=ax)
        b.set(ylabel="No. of Students", xlabel="Campus of Students", title="Total Students in Each Campus")
        self.plot(figure)
        
    def program_at_each_campus(self):
        figure = Figure(figsize=(10, 10))
        ax = figure.add_subplot(111)
        b = sns.countplot(x="CAMPUS", hue="PROG_CODE", data = self.get_data(), ax=ax)
        b.set(ylabel="No. of Students", xlabel="Campus", title="Degree Programs offered at Each Campus")
        self.plot(figure)
    
    def performance_at_each_campus(self):
        figure = Figure(figsize=(10, 10))
        ax = figure.add_subplot(111)
        b = sns.boxplot(x="CAMPUS", y="CGPA", data = self.get_data(), ax=ax)
        b.set(ylabel="CGPA of Students", xlabel="Campus", title="Performance of Students at Each Campus")
        self.plot(figure)            
        
    def gender_of_students(self, f=None):
        figure = Figure(figsize=(10, 10))
        ax = figure.add_subplot(111)
        b = sns.countplot(x="GENDER", data = self.get_data(f), ax=ax)
        if(f):
            title = "Total No. of Female & Male Students at " + f  + " Campus"                                                                                                      
            b.set(ylabel="No. of Students", xlabel="Gender of Students", title=title)
        else:                                                                                                              
            b.set(ylabel="No. of Students", xlabel="Gender of Students", title="Female & Male Students")
        self.plot(figure)
        
    def students_in_each_program(self, f=None):
        figure = Figure(figsize=(10, 10))
        ax = figure.add_subplot(111)
        b = sns.countplot(x="PROG_CODE", data = self.get_data(f), ax=ax)
        if(f):
            title = "Total Students in Each Degree Program at " + f + " Campus" 
            b.set(ylabel="No. of Students", xlabel="Degree Program", title=title)
        else:
            b.set(ylabel="No. of Students", xlabel="Degree Program", title="Total Students in Each Degree Program")
        self.plot(figure)
        
    def secondary_background(self, f=None):
        figure = Figure(figsize=(10, 10))
        ax = figure.add_subplot(111)
        b = sns.countplot(x="SECONDARY", data = self.get_data(f), ax=ax)
        if (f):
            title = "Secondary education background of Students at " + f + " Campus"
            b.set(ylabel="No. of Students", xlabel="Secondary Education", title=title)
        else:
            b.set(ylabel="No. of Students", xlabel="Secondary Education", title="Secondary education background of Students")
        self.plot(figure)

    def hig_secondary_background(self, f=None):
        figure = Figure(figsize=(10, 10))
        ax = figure.add_subplot(111)
        b = sns.countplot(x="HIGHER_SECONDARY", data = self.get_data(f), ax=ax)
        if (f):
            title = "Higher education background of Students at " + f + " Campus"
            b.set(ylabel="No. of Students", xlabel="Higher Secondary Education", title=title)
        else:
            b.set(ylabel="No. of Students", xlabel="Higher Secondary Education", title="Higher education background of Students")
        self.plot(figure)
        
    def students_in_each_city(self, f=None):
        figure = Figure(figsize=(10, 10))
        ax = figure.add_subplot(111)
        b = sns.countplot(x="CITY", data = self.get_data(f), ax=ax)
        if (f):
            title = "City of Students at " + f + " Campus"
            b.set(ylabel="No. of Students", xlabel="City", title=title)
        else:
            b.set(ylabel="No. of Students", xlabel="City", title="City of Students")
        self.plot(figure)
        

    #------------------------------Model Building Functions-------------------------------------
        
    def lr__(self):
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            print("Build Linear Regression Model")
            acc = linear_regression(self.training_data)
            acc *= 100
            self.accuracy.set(" " + str(round(acc, 2)) + " %")
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")
        
    def gnb__(self):
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            print("Build Gaussian Naive Bayes Model")
            acc = gnb(self.training_data)
            acc *= 100
            self.accuracy.set(" " + str(round(acc, 2)) + " %")        
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")
        
    def logR__(self):
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            print("Build Logistic Regression Model")
            acc = logistic_regression(self.training_data)
            acc *= 100
            self.accuracy.set(" " + str(round(acc, 2)) + " %")
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")                
        
    def knn__(self):
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            print("Build KNN Model")
            acc = knn(self.training_data)
            acc *= 100
            self.accuracy.set(" " + str(round(acc, 2)) + " %")
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")
            
    def dt__(self):
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            print("Build Decision Tree Model")
            acc = decisiontree(self.training_data)
            acc *= 100
            self.accuracy.set(" " + str(round(acc, 2)) + " %")
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")
            
    def rf__(self):
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            print("Build Random Forest Model")
            acc = randomforest(self.training_data)
            acc *= 100
            self.accuracy.set(" " + str(round(acc, 2)) + " %")
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")
            
    def svm__(self):
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            print("Build SVM Model")
            acc = svm(self.training_data)
            acc *= 100
            self.accuracy.set(" " + str(round(acc, 2)) + " %")
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")
            
    #-----------------------------------Predictive Modeling Functions---------------------------
            
    def P_1st(self):
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            self.test_sem = predict_1st_semester(self.training_data, self.testing_data)
            col = []
            for i in self.test_sem.columns: col.append(i)
            self.table_frame3(col)
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")
        
    def P_2nd(self):
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            self.test_sem = predict_2nd_semester(self.training_data, self.testing_data)
            col = []
            for i in self.test_sem.columns: col.append(i)
            self.table_frame3(col)
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")
            
    def P_3rd(self):
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            self.test_sem = predict_3rd_semester(self.training_data, self.testing_data)
            col = []
            for i in self.test_sem.columns: col.append(i)
            self.table_frame3(col)
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")
            
    def P_4th(self):
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            self.test_sem = predict_4th_semester(self.training_data, self.testing_data)
            col = []
            for i in self.test_sem.columns: col.append(i)
            self.table_frame3(col)
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")
            
    def P_5th(self):
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            self.test_sem = predict_5th_semester(self.training_data, self.testing_data)
            col = []
            for i in self.test_sem.columns: col.append(i)
            self.table_frame3(col)
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")
            
    def P_6th(self):
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            self.test_sem = predict_6th_semester(self.training_data, self.testing_data)
            col = []
            for i in self.test_sem.columns: col.append(i)
            self.table_frame3(col)
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")
            
    def P_7th(self):
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            self.test_sem = predict_7th_semester(self.training_data, self.testing_data)
            col = []
            for i in self.test_sem.columns: col.append(i)
            self.table_frame3(col)
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")
            
    def P_8th(self):    
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            self.test_sem = predict_8th_semester(self.training_data, self.testing_data)
            col = []
            for i in self.test_sem.columns: col.append(i)
            self.table_frame3(col)
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")
            
        
    #---------------------------------Functions for GUI Buttons-------------------------------
    
    def view_student_data(self):      
        if (self.studentDataImport & self.studentDataTransform & self.studentDataClean):
            col = []
            for i in self.training_data.columns: col.append(i)
            self.table_frame1(col, self.training_data)
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
        elif (self.studentDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Student Data'")
    
    def view_course_data(self):
        if (self.courseDataImport & self.courseDataTransform & self.courseDataClean):
            col = []
            for i in self.CourseData.columns: col.append(i)
            self.table_frame1(col, self.CourseData)
        elif (self.courseDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Course Data'")
        elif (self.courseDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Course Data'")
        elif (self.courseDataClean == False):
            messagebox.showinfo("Error", "Please, Clean 'Course Data'")
        
    def clear_data(self):
        choice = messagebox.askyesno('Clear Data', 'Do you want to remove data?')
        if choice:
            self.CourseData1 = pd.DataFrame()
            self.CourseData2 = pd.DataFrame()
            self.CourseData = pd.DataFrame()
            self.StudentData = pd.DataFrame()
            self.SemesterData = pd.DataFrame()
            self.training_data = pd.DataFrame()
            self.testing_data = pd.DataFrame()
            self.test_sem = pd.DataFrame() 
            self.accuracy.set('')
        
            if (self.areaFrame1):
                self.areaFrame1 = False
                self.frame1area.destroy()               
            
            if (self.areaFrame3):
                self.areaFrame3 = False
                self.frame3area.destroy()   
                
            if(self._canvas__):
                self._canvas__ = False
                self.frame_vis.destroy()
                                
            messagebox.showinfo("Alert Message", "Dataset Deleted!")
        else:
            return
    
    def exit(self):
        choice = messagebox.askyesno('Exit','Do you want to exit?')
        if choice:
            root.destroy()
        else:
            return
        
    #----------------------------------Helper Functions-------------------------------
        
    def get_data(self, filter_= None):
        data = pd.merge(self.StudentData, self.SemesterData, on="STUDENT_ID")
        data = data.loc[data['GENDER'] != '-']
        data["WARNING"] = data["WARNING"].fillna(0)
        if (filter_):
            data = data.loc[data["CAMPUS"] == filter_]
        return data

    def plot(self, figure):
        if (self.studentDataImport & self.studentDataTransform):
            if(self._canvas__):
                self.frame_vis.destroy()
            self.frame_vis = Frame(self.frame2, bg="white") 
            self.frame_vis.place(x=1, y=50, width=425, height=346)
            self.canvas = FigureCanvasTkAgg(figure, master=self.frame_vis)
            self.canvas.draw()
            self.canvas.get_tk_widget().pack()
            self._canvas__ = True
        elif (self.studentDataImport == False):
            messagebox.showinfo("Error", "Please, Import 'Student Data'")
        elif (self.studentDataTransform == False):
            messagebox.showinfo("Error", "Please, Apply transformation on 'Student Data'")
    
    def table_frame1(self, col_names, data):
        if (self.areaFrame1):
            self.frame1area.destroy()
        self.frame1area = ttk.Treeview(self.frame1, columns=col_names, xscrollcommand=self.f1_scroll_x.set, yscrollcommand=self.f1_scroll_y.set)
        for x in col_names:
            self.frame1area.heading(x, text=x)
            self.frame1area.column(x, width=80)
        self.frame1area['show'] = 'headings'
        self.frame1area.pack(fill=BOTH, expand=1)
        self.f1_scroll_x.config(command=self.frame1area.xview)
        self.f1_scroll_y.config(command=self.frame1area.yview)
        self.frame1area.pack(fill=BOTH, expand=1)
        data = list(data.itertuples(index=False, name=None))
        for row in data:
            self.frame1area.insert('', END, values=row)     
        self.areaFrame1 = True
        
    def table_frame3(self, col_names):
        if (self.areaFrame3):
            self.frame3area.destroy()
        self.frame3area = ttk.Treeview(self.frame3, columns=col_names, xscrollcommand=self.f3_scroll_x.set, yscrollcommand=self.f3_scroll_y.set)
        for x in col_names:
            self.frame3area.heading(x, text=x)
            self.frame3area.column(x, width=80)
        self.frame3area['show'] = 'headings'
        self.frame3area.pack(fill=BOTH, expand=1)
        self.f3_scroll_x.config(command=self.frame3area.xview)
        self.f3_scroll_y.config(command=self.frame3area.yview)
        self.frame3area.pack(fill=BOTH, expand=1)
        data = list(self.test_sem.itertuples(index=False, name=None))
        for row in data:
            self.frame3area.insert('', END, values=row)
        self.areaFrame3 = True
        
    def not_defined(self):
        messagebox.showinfo("Alert Message", "Function not Defined")

In [395]:
root = Tk()
obj = Predictive_Analytics(root)
root.mainloop()

In [392]:
StudentData = pd.read_csv("C:\\Users\\Areeka Aijaz\\Desktop\\FYP_\\Cleaned_Student_Data.csv", encoding= 'unicode_escape')
SemesterData = pd.read_csv("C:\\Users\\Areeka Aijaz\\Desktop\\FYP_\\Cleaned_Semester_Data.csv", encoding= 'unicode_escape')

In [371]:
training_data = pd.read_csv("C:\\Users\\Areeka Aijaz\\Desktop\\FYP_\\Training_Data.csv")
testing_data =  pd.read_csv("C:\\Users\\Areeka Aijaz\\Desktop\\FYP_\\Testing_Data.csv")